It's better to have Whisper in a separate environment from ESPnet. Also, we use `faster-whisper` as it is not only faster but more stable than the original.
```shell
conda create -n whisper python=3.10
conda activate whisper
pip install faster-whisper
```
before running this.

You also need to set the env variable LD_LIBRARY_PATH:

```shell
conda activate whisper
env_loc=$(conda env list | grep '*' | awk '{print $3}' | tr -d '\n')
activator="${env_loc}/etc/conda/activate.d/cuda_activate.sh"
echo export LD_LIBRARY_PATH=`python3 -c 'import os; import nvidia.cublas.lib; import nvidia.cudnn.lib; import torch; print(os.path.dirname(nvidia.cublas.lib.__file__) + ":" + os.path.dirname(nvidia.cudnn.lib.__file__) + ":" + os.path.dirname(torch.__file__) +"/lib")'` > "${activator}"
chmod +x "${activator}"
conda deactivate
conda activate whisper
```

In [1]:
from pathlib import Path
import os
import faster_whisper
from tqdm import tqdm
import jiwer
import re

PWD = %pwd
PWD = Path(PWD)
prosody_dir = PWD.parent
outputs_dir = PWD / 'outputs'
os.makedirs(outputs_dir, exist_ok=True)
asr_dir = outputs_dir / 'CSS10' / 'hungarian'
os.makedirs(asr_dir, exist_ok=True)
jets_dir = prosody_dir / 'outputs' / 'tts_train_jets_raw_phn_tacotron_g2p_en_no_space/CSS10/hungarian'
data_dir = (prosody_dir / '../../datasets/CSS10/hungarian/').resolve()

In [2]:
import os
ld_lib_path = os.environ['LD_LIBRARY_PATH']
assert 'cublas' in ld_lib_path and 'cudnn' in ld_lib_path

In [55]:
faster_whisper.available_models()

['tiny.en',
 'tiny',
 'base.en',
 'base',
 'small.en',
 'small',
 'medium.en',
 'medium',
 'large-v1',
 'large-v2',
 'large-v3',
 'large',
 'distil-large-v2',
 'distil-medium.en',
 'distil-small.en',
 'distil-large-v3']

In [3]:
model = faster_whisper.WhisperModel("large-v2", device='cuda', compute_type='float16')
asr_dir = asr_dir / 'large-v2'
os.makedirs(asr_dir, exist_ok=True)

In [ ]:
hungarian_letters = set()
with open(data_dir / 'transcript.txt') as f:
    for line in f:
        hungarian_letters |= set(line.split('|')[2].lower())

In [6]:
''.join(sorted(hungarian_letters))

' !,-.:;?abcdefghijklmnoprstuvwxyzáéíóöúüőű'

In [4]:
def normalize_hungarian(text):
    text = text.lower()
    text = re.sub(r"[¡¿…«»“”?!',.:;–-]", ' ', text)
    text = re.sub(r'[^abcdefghijklmnoprstuvwxyzáéíóöúüőű ]', '', text)
    text = ' '.join(text.strip().split())
    return text

In [5]:
transcript_file = data_dir / 'transcript_normalized.txt'
if not transcript_file.exists():
    with open(data_dir / 'transcript.txt') as f:
        with open(transcript_file, 'w') as norm_f:
            for line in f:
                filename, _, transcript, _ = line.split('|')
                if transcript:
                    transcript = normalize_hungarian(transcript)
                    norm_f.write(f'{filename}|{transcript}\n')

def get_transcripts():
    transcripts = {}
    with open(transcript_file) as f:
        for line in f:
            filename, transcript = line.strip().split('|', maxsplit=1)
            transcripts[filename] = transcript
    return transcripts

transcripts = get_transcripts()

In [6]:
filenames = list(transcripts.keys())

In [6]:
whisper_kwargs = {
    # 'suppress_tokens': suppress_tokens,
    # 'temperature': 0.0,
    # 'condition_on_previous_text': False,
    'prepend_punctuations': '',
    'append_punctuations': '',
}

In [7]:
from num2words import num2words
import numpy as np
def whisper_transcribe(filepath, kwargs=whisper_kwargs):
    segments, _ = model.transcribe(filepath, language='hu', **kwargs)
    text = ' '.join(segment.text for segment in segments)
    pads = 1
    while not text:  # whisper sometimes randomly fails to produce anything
        audio = faster_whisper.audio.decode_audio(filepath)
        audio_pad = np.pad(audio, (pads * 100, 0), mode='constant', constant_values=0)
        segments, _ = model.transcribe(audio_pad, language='hu', **kwargs)
        text = ' '.join(segment.text for segment in segments)
        pads += 1
        if pads == 10:
            break
    splits = re.split(r'(\d+)', text)
    for i in range(len(splits)):
        if re.fullmatch(r'\d+', splits[i]):
            splits[i] = num2words(splits[i], lang='hu')
    text = ''.join(splits)
    return text

In [32]:
whisper_transcribe(data_dir / filenames[3052])

NameError: name 'filenames' is not defined

In [8]:
def run_asr(filenames, audio_dir, asr_result_path, kwargs=whisper_kwargs):
    with open(asr_result_path, 'w') as f:
        for filename in tqdm(filenames):
            wav_path = audio_dir / filename
            text = whisper_transcribe(wav_path, kwargs)
            text = normalize_hungarian(text)
            if not text:  
                text = whisper_transcribe(wav_path, kwargs)
                text = normalize_hungarian(text)
                if not text:
                    print(f'{filename}')
            f.write(f'{filename}|{text}\n')
            

In [9]:
audio_dir = Path('/home/perry/PycharmProjects/present/prosody/outputs/tts_train_jets_raw_phn_tacotron_g2p_en_no_space/hungarian')
def get_phone_cer(phone):
    print('\n', phone)
    phone_dir = audio_dir / phone
    fnames = os.listdir(phone_dir)
    avg_cer = 0
    for fname in fnames:
        print(fname)
        book = fname.rsplit('_', 1)[0]
        orig = transcripts[f'{book}/{fname}']
        trans = normalize_hungarian(whisper_transcribe(phone_dir / fname))
        print(orig)
        print(trans)
        cer = jiwer.cer(orig.replace(' ', ''), trans.replace(' ', ''))
        print(cer)
        avg_cer += cer
    avg_cer /= len(fnames)
    return avg_cer

In [10]:
get_phone_cer('u')


 u
egri_csillagok_2932.wav
kardjuk damaszkuszban készült vértjük derbendi acél lándzsájuk hindosztáni mesterkovácsoktól való ágyúikat európa legjobb öntői alkotják meg puskaporuk golyójuk fegyverük mérhetetlen és megszámlálhatatlan
kárgyi gámasz kuszban készült vérjük derbindé atszáll lánzsíjük hindustáni mestyárkó vacsoktól való ágyúikat európa legjobb öntői alkotjak meg puskáparók golyójuk fejverjük mérhetetlenné s megszámolhatatlan
0.18579234972677597
egri_csillagok_4228.wav
a szentségben amit itt látunk tudjuk hogy az élő jézus van jelen velünk van boruljunk le és imádkozzunk
a szentségben ami itt látunk tudjuk hogy az elo jézus van jelen velünk van boruljunk le és imádkozunk
0.046511627906976744
egri_csillagok_2315.wav
tudod uram szegények vagyunk hát esténkint halásznunk kell hanem az éjjel nemcsak halat fogtunk ahogy kihúzzuk a hálót valami megcsillanik benne
tudod múlvam szegények vagyunk hát esténként halasznunk kell hanem az eljönnem csak halatfotónk ahogy hihuzuka halott va

0.1392626162004982

In [48]:
phones = 'aː ɑ ɑː eː ɛ i iː o oː ø øː u uː y yː b bː c cː d dː dzː dʒ f h j k kː l m n p pː r s t tː ts tsː tʃ tʃː v z ɟ ɟː ɡ ɡː ɲ ʃ ʎ ʒ'.split()
phone_cers = []
for p in phones:
    phone_cers.append(get_phone_cer(p))


 aː
egri_csillagok_4342.wav
az asszony csapást csapás után oszt a felnyomakodó törökre míg végre lándzsaszúrás találja s elhanyatlik le a bástyáról az állványra nincs már aki elrántsa
az a szönycsápás csápás jután oszt a felnyomagadó törökre míg végre láncsasz a rastalolja se lennetlik le a bási rol az alvanyra nincs már aki elrannsak
0.20454545454545456
egri_csillagok_2407.wav
de ez úgy hányja veti magát mint a szultán lova noha az is többet tud nálánál hanem bezzeg mikor egy magánál nagyobb fát lát hajlong mint a kender szélfúváskor
de ez úgy nani vették magát mint a szultánlóva nyolga az is többet tudná lennol nálim bezzeg mikor ezs maganol názsopfátlát nálon mint a kender szél füveskor
0.2595419847328244
egri_csillagok_3826.wav
aztán a dárdát ismét bevonják és a fényszárny a másik bástyánál lövell ki az éjszakába dobó a nyugati kapunál megáll az őr tiszteleg
aztán ad arra hogy ismét bevonjuk egy szafényszorny a másik básiánól lovil aki az éjszakába dobó anyugati kábanol megöl az ú

In [49]:
for p, cer in zip(phones, phone_cers):
    print(p, cer)

aː 0.23954429794124452
ɑ 0.4813129961644813
ɑː 0.2013768367465001
eː 0.20219868362075066
ɛ 0.2411278100530437
i 0.17282876613020157
iː 0.15661561785642375
o 0.22319163273078838
oː 0.19832817366637925
ø 0.2654971234097723
øː 0.23386111495791595
u 0.1883971675979104
uː 0.23231342524526802
y 0.14045214045214047
yː 0.3324092391475007
b 0.19355877616747183
bː 0.21742287272205482
c 0.33374799627571244
cː 0.2608029772978227
d 0.12333309681382654
dː 0.21917087542087543
dzː 0.18559509484269443
dʒ 0.253780143866654
f 0.16011242122828975
h 0.2595737789869154
j 0.19678891876874163
k 0.18053120603807327
kː 0.22212747590548532
l 0.20654748727305325
m 0.1889216757741348
n 0.13310567799204162
p 0.1967065412919414
pː 0.21298418639862052
r 0.180021455496999
s 0.19699048048470083
t 0.23448015114681783
tː 0.2812591951856041
ts 0.27485265110756163
tsː 0.19965638412364983
tʃ 0.1907293352123458
tʃː 0.2317497403946002
v 0.19365138881449342
z 0.16499181030756324
ɟ 0.1974945188718503
ɟː 0.223015873015873
ɡ 0.19

In [46]:
import logging
logging.basicConfig(level=logging.WARNING)

In [116]:
gt_dir = data_dir
gt_asr_path = asr_dir / 'gt_result.txt'

jets_asr_path = asr_dir / 'jets_result.txt'

In [118]:
run_asr(filenames, gt_dir, gt_asr_path)

100%|██████████| 4514/4514 [50:06<00:00,  1.50it/s]  


In [117]:
import time
time.sleep(700)
run_asr(filenames, jets_dir, jets_asr_path)

100%|██████████| 4514/4514 [54:04<00:00,  1.39it/s]  


In [119]:
def eval_cer(transcripts, asr_result_path, cer_path):
    with open(cer_path, 'w') as cer_file:
        cer_file.write('wav_file,gt_len,cer\n')
        with open(asr_result_path) as f:
            for line in f:
                wav_file, asr_output = line.strip('\n').split('|', maxsplit=1)
                transcript = transcripts[wav_file]
                transcript_nospace = transcript.replace(' ', '')
                asr_nospace = asr_output.replace(' ', '')
                gt_len = len(transcript)
                cer = jiwer.cer(truth=transcript_nospace, hypothesis=asr_nospace)
                cer_file.write(f'{wav_file},{gt_len},{cer}\n')

In [120]:
gt_cer_path = asr_dir / 'gt_cer.csv'
eval_cer(transcripts=transcripts, asr_result_path=gt_asr_path, cer_path=gt_cer_path)

In [121]:
jets_cer_path = asr_dir / 'jets_cer.csv'
eval_cer(transcripts=transcripts, asr_result_path=jets_asr_path, cer_path=jets_cer_path)